In [ ]:
!pip install geopandas rasterio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.5/21.5 MB 10.0 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
import pandas as pd
import geopandas as gpd
import os
import rasterio
import rasterio.features
import rasterio.transform
from google.colab import drive


# Define the folder containing the CSV files
csv_folder_path = '/content/drive/MyDrive/sam'

# Load the California shapefile
shapefile_path = '/content/drive/MyDrive/EE ML PFAS/Raw_Data/Mapping/California_Shapefile/California.shp'
ca_shapefile = gpd.read_file(shapefile_path)
ca_shapefile = ca_shapefile.to_crs('EPSG:4326')

# Function to convert DataFrame to GeoDataFrame
def convert_to_gdf(df, lat_col='Latitude', lon_col='Longitude'):
    return gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df[lon_col], df[lat_col]), crs='EPSG:4326')

# Function to plot and save GeoDataFrame as TIFF using rasterio
def plot_and_save_tiff(gdf, title, output_path, bbox, width=1000, height=1000):
    if gdf.empty or gdf.geometry.is_empty.all():
        print(f"No valid geometries found for file: {output_path}")
        return
    # Dedent this line to align with 'if' statement
    transform = rasterio.transform.from_bounds(*bbox, width, height)
    out_shape = (height, width)

    # Rasterize the GeoDataFrame
    out_image = rasterio.features.rasterize(
        [(geom, 1) for geom in gdf.geometry if geom.is_valid],
        out_shape=out_shape,
        transform=transform,
        fill=0,
        all_touched=True,
        dtype='uint8'
    )

    # Save the rasterized image as a TIFF file
    with rasterio.open(output_path, 'w', driver='GTiff',
                       height=out_shape[0], width=out_shape[1],
                       count=1, dtype='uint8',
                       crs=gdf.crs, transform=transform) as dst:
        dst.write(out_image, 1)



# Process each CSV file in the folder
for filename in os.listdir(csv_folder_path):
    if filename.endswith('.csv'):
        # Load CSV file
        file_path = os.path.join(csv_folder_path, filename)
        soil_data = pd.read_csv(file_path)

        # Convert to GeoDataFrame
        soil_gdf = convert_to_gdf(soil_data)

        # Reproject to match California shapefile CRS
        soil_gdf = soil_gdf.to_crs(ca_shapefile.crs)

        # Perform spatial intersection
        soil_ca = gpd.sjoin(soil_gdf, ca_shapefile, how='inner', op='intersects')

        # Check for invalid or missing latitude values
        invalid_latitudes = soil_ca[
            (soil_ca.geometry.y < -90) | (soil_ca.geometry.y > 90) | soil_ca.geometry.y.isna()
        ]

        # If there are invalid latitudes, print a message and continue
        if not invalid_latitudes.empty:
            print(f"Invalid or missing latitudes found in file: {filename}")
            continue

        # Define output path for TIFF file
        tiff_output_path = os.path.join('/content/drive/MyDrive/EE ML PFAS/Raster_Data/Environmental_Quality/Soil_Quality/Soil_Quality_Raster_Data ', f'{filename.replace(".csv", "")}.tif')

       # Get the bounding box from the CA shapefile
        bbox = ca_shapefile.total_bounds

        # Plot and save TIFF using rasterio
        plot_and_save_tiff(soil_ca, f'Soil Data in California - {filename}', tiff_output_path, bbox)

        print("Processing complete.")

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3473: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if (await self.run_code(code, result,  async_=asy)):
/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3473: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if (await self.run_code(code, result,  async_=asy)):
/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3473: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if (await self.run_code(code, result,  async_=asy)):


No valid geometries found for file: /content/drive/MyDrive/EE ML PFAS/Raster_Data/Environmental_Quality/Soil_Quality/Soil_Quality_Raster_Data /top_soil_year_quarter_2009_1.tif
Processing complete.
No valid geometries found for file: /content/drive/MyDrive/EE ML PFAS/Raster_Data/Environmental_Quality/Soil_Quality/Soil_Quality_Raster_Data /upper_soil_year_quarter_2007_2.tif
Processing complete.
No valid geometries found for file: /content/drive/MyDrive/EE ML PFAS/Raster_Data/Environmental_Quality/Soil_Quality/Soil_Quality_Raster_Data /upper_soil_year_quarter_2007_3.tif
Processing complete.


/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3473: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if (await self.run_code(code, result,  async_=asy)):


Processing complete.


/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3473: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if (await self.run_code(code, result,  async_=asy)):


...

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3473: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if (await self.run_code(code, result,  async_=asy)):


No valid geometries found for file: /content/drive/MyDrive/EE ML PFAS/Raster_Data/Environmental_Quality/Soil_Quality/Soil_Quality_Raster_Data /top_soil_year_quarter_2007_2.tif
Processing complete.


/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3473: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if (await self.run_code(code, result,  async_=asy)):


No valid geometries found for file: /content/drive/MyDrive/EE ML PFAS/Raster_Data/Environmental_Quality/Soil_Quality/Soil_Quality_Raster_Data /upper_soil_year_quarter_2009_1.tif
Processing complete.
No valid geometries found for file: /content/drive/MyDrive/EE ML PFAS/Raster_Data/Environmental_Quality/Soil_Quality/Soil_Quality_Raster_Data /top_soil_year_quarter_2007_3.tif
Processing complete.


/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3473: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if (await self.run_code(code, result,  async_=asy)):
